In [1]:
%matplotlib inline
import copy
import qutip
from qutip import Qobj
import numpy as np
import math, cmath
import matplotlib.pyplot as plt
from tqdm import tqdm

from stateobj import Physics
import utilities as use
import circuits as qc

import os

from IPython.display import Markdown, display

In [2]:
TIMESTEPS = 5000
TIMEDELTA = 1e-01
OMEGA = 0.5  # Strength of Interaction

D = 5

p = Physics(dimension=D, interaction_strength=OMEGA, interaction_time=TIMEDELTA)

th = OMEGA * TIMEDELTA
alpha = complex(1/math.sqrt(2), 0)
beta = cmath.sqrt(1 - alpha**2)
phi = np.pi/2
# Phase shifts
delta1 = 0
delta2 = -phi

In [3]:
eta = use.create_ancilla_qobj(alpha, beta, phi)
rho1 = use.create_system_qobj('fock', n_dims=D)
rho2 = use.create_system_qobj('fock', n_dims=D)

display(qutip.tensor(rho1, rho2))

Quantum object: dims = [[5, 5], [5, 5]], shape = (25, 25), type = oper, isherm = True
Qobj data =
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0.

In [6]:
# Ancilla dephasing gate
class DephasingGate(qc.Gate):
    # Unitary evolution Matrix
    U = Qobj([[1, 0, 0], [0, 1, 0], [0, 0, cmath.exp(-1j*delta2)]])
    
    def evolve(self, system):
        return self.U * system * self.U.dag()
    
class MeasureGate(qc.Gate):
    def __init__(self, *args, partial_system=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.partial_system = partial_system

    def evolve(self, system):
        display(Markdown(f"{self.name.upper()} measure"))
        if self.partial_system is not None:
            traced_out_system = system.ptrace(self.partial_system)
        else:
            traced_out_system = system
        display(traced_out_system)
        return system
    
class FirstTimeEvolutionGate(qc.Gate):
    # This acts only on the first system
    def evolve(self, system):
        U = (1j * th * p.V1).expm()
        return U * system * U.dag()
    
class SecondTimeEvolutionGate(qc.Gate):
    def evolve(self, system):
        # This acts only on the second system
        U = (1j * th * p.V2).expm()
        return U * system * U.dag()

class TraceOutGate(qc.Gate):
    def __init__(self, controller, sys_to_keep=0, *args, **kwargs):
        super().__init__(controller, *args, **kwargs)
        self.systems = sys_to_keep

    def evolve(self, system):
        return system.ptrace(self.systems)

    def inject(self, step, system):
        for lane in self.lanes:
            if lane not in self.systems:
                self.controller.lanes[lane].system_evolution[step + 1] = None
            else:
                self.controller.lanes[lane].system_evolution[step + 1] = system

    
class MasterEquationGate(qc.Gate):
    
    def dissipator(self, X: Qobj, system: Qobj):
            sandwich = X * system * X.dag()
            comm = qutip.commutator(X.dag()*X, system, kind='anti')
            return sandwich - 1/2 * comm
        
    def master_equation(self, system):
        ga = 2*alpha**2
        gb = (beta)**2*(1-np.cos(phi))
        # Bosonic Operators
        C = p.C
        Cp = p.Cp
        S = p.S
        Sd = p.S.dag()
        first_line = 0.5*self.dissipator(qutip.tensor(C, C) - 2*qutip.tensor(S, Sd), system) 
        first_line += self.dissipator(qutip.tensor(C, S) + qutip.tensor(S, Cp), system)
        second_line = 0.5*self.dissipator(qutip.tensor(Cp, Cp) - 2*qutip.tensor(Sd, S), system)
        second_line += self.dissipator(qutip.tensor(Cp, Sd) + qutip.tensor(Sd, C), system)
        return ga * first_line + gb * second_line
    
    def evolve(self, system):
        system_variation = self.master_equation(system)
        return system + system_variation
        


In [8]:
class QuantumCircuit(qc.Circuit):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.gates = [
            DephasingGate(self, [1], name='u1', control_lane=0),
            # MeasureGate(self, [1], name='mi'),
            FirstTimeEvolutionGate(self, [2, 1], name='U1'),
            # MeasureGate(self, [1], name='ms', partial_system=2),
            SecondTimeEvolutionGate(self, [2, 1], name='U2'),
            TraceOutGate(self, lanes=[2, 1], sys_to_keep=[0, 1], name='tr'),
        ]
        self.lanes = [
            qc.Lane(self, +1, name='c1'),
            qc.Lane(self, eta, name='eta'),
            qc.Lane(self, qutip.tensor(rho1, rho2), name='rho'),
        ]
a = QuantumCircuit(n_lanes=3, logging=True)
a.draw()
a.run()
a.draw()

         1   2   3   4  
c1  -----==----------------
eta -----u1--U1--U2--tr----
rho ---------U1--U2--tr----

Step 1 - Gate u1 acting on [1]
Used lanes: [eta: Quantum System [3]x[3] (not entangled)]
________________________________________

Step 2 - Gate U1 acting on [2, 1]
Used lanes: [rho: Quantum System [5, 5]x[5, 5] (entangled), eta: Quantum System [3]x[3] (not entangled)]
________________________________________

Step 3 - Gate U2 acting on [2, 1]
Used lanes: [rho: Quantum System [5, 5]x[5, 5] (entangled), eta: Quantum System [3]x[3] (not entangled)]
________________________________________

Step 4 - Gate tr acting on [2, 1]
Used lanes: [rho: Quantum System [5, 5]x[5, 5] (entangled), eta: Quantum System [3]x[3] (not entangled)]
________________________________________

         1   2   3   4  
c1  -----==----------------
eta -----u1--U1--U2--tr----
rho ---------U1--U2--tr-   



In [ ]:
a.lanes[1].system_evolution

In [ ]:
circuit = qc.Circuit(n_lanes=3, logging=True)
circuit.populate_lane(0, -1, 'c1')
circuit.populate_lane(1, eta, 'eta')
circuit.populate_lane(2, qutip.tensor(rho1, rho2), 'rho')

#circuit.add_gate(MeasureGate([1], name='m2'))
#circuit.add_gate(MasterEquationGate([1], name='meq'))
circuit.add_gate(TraceOutGate(sys_to_keep=[0, 1], lanes=[2, 1], name='tr'))
circuit.add_gate(SecondTimeEvolutionGate([2, 1], name='U2'))
circuit.add_gate(MeasureGate([1], name='ms', partial_system=2), 0)
circuit.add_gate(FirstTimeEvolutionGate([2, 1], name='U1'))
circuit.add_gate(MeasureGate([1], name='mi'), 0)
circuit.add_gate(DephasingGate([1], name='u1', control_lane=0))
circuit.draw()

In [ ]:
for t in range(10):
    print(f"T = {t}")
    # Inject a new Ancilla in the circuit
    circuit.populate_lane(1, eta, 'eta')
    if t == 3:
        # Switch the phase controller
        circuit.update_lane(0, system=1)
    if t == 5:
        circuit.add_gate(MeasureGate([1], name='mf', partial_system=2), position=4)
    circuit.draw()
    circuit.run()
    print(f"{'='*40}")

In [ ]:
display(circuit.lanes[2].system.ptrace(0))
display(circuit[2].ptrace(0))